<a href="https://www.kaggle.com/code/rebeccapringle/heart-failure-prediction-dl-classification?scriptVersionId=135124524" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Importing Libraries

In [1]:
import numpy as np
import pandas as pd

# Import Datasets

In [2]:
df = pd.read_csv('/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

# Size and check nulls

In [3]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB
None


In [4]:
from collections import Counter
print('Classes and number of values in the dataset',Counter(df['DEATH_EVENT']))

Classes and number of values in the dataset Counter({0: 203, 1: 96})


# Split and prepare dataset

In [5]:
y = df["DEATH_EVENT"]

X = df[['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking','time']]
X  = pd.get_dummies(X)

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# Normalise numerical columns

In [7]:
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([("numeric", Normalizer(), ['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time'])])

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

# Encode category columns

In [8]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# labels for classification
le = LabelEncoder()

y_train = le.fit_transform(y_train.astype(str))
y_test = le.transform(y_test.astype(str))

In [9]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

# Build model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
model = Sequential()

model.add(InputLayer(input_shape = (X_train.shape[1],)))

In [11]:
from tensorflow.keras.layers import Dense

model.add(Dense(12, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train model and evaluate

In [12]:
# training and evaluation
model.fit(X_train, y_train, epochs = 50, batch_size = 2, verbose=1)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print("Loss", loss, "Accuracy:", acc)

Epoch 1/50
105/105 [==============================] - 1s 2ms/step - loss: 0.6413 - accuracy: 0.6746
Epoch 2/50
105/105 [==============================] - 0s 2ms/step - loss: 0.6355 - accuracy: 0.6746
Epoch 3/50
105/105 [==============================] - 0s 2ms/step - loss: 0.6345 - accuracy: 0.6746
Epoch 4/50
105/105 [==============================] - 0s 2ms/step - loss: 0.6321 - accuracy: 0.6746
Epoch 5/50
105/105 [==============================] - 0s 2ms/step - loss: 0.6319 - accuracy: 0.6746
Epoch 6/50
105/105 [==============================] - 0s 2ms/step - loss: 0.6316 - accuracy: 0.6746
Epoch 7/50
105/105 [==============================] - 0s 2ms/step - loss: 0.6321 - accuracy: 0.6746
Epoch 8/50
105/105 [==============================] - 0s 2ms/step - loss: 0.6334 - accuracy: 0.6746
Epoch 9/50
105/105 [==============================] - 0s 1ms/step - loss: 0.6323 - accuracy: 0.6746
Epoch 10/50
105/105 [==============================] - 0s 1ms/step - loss: 0.6317 - accuracy: 0.6746

In [13]:
y_estimate = np.argmax(model.predict(X_test, verbose = 0), axis = 1)

y_true = np.argmax(y_test, axis = 1)

from sklearn.metrics import classification_report

cr = classification_report(y_true, y_estimate)
print(cr)

              precision    recall  f1-score   support

           0       0.69      1.00      0.82        62
           1       0.00      0.00      0.00        28

    accuracy                           0.69        90
   macro avg       0.34      0.50      0.41        90
weighted avg       0.47      0.69      0.56        90



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
